In [7]:
import pandas as pd
import numpy as np
import json
import ast


df1 = pd.read_csv('Kickstarter001.csv')

photo_link = []
country = []
state = []
town = []
loc_type = []
project_url = []
reward_url =[]
list_category = []
list_subcategory = []

for i in range(len(df1)):
    photo_dict =df1.iloc[i].photo
    photo_link.append(ast.literal_eval(photo_dict)['full'])
    
for i in range(len(df1)):
    try:
        category, subcategory  = json.loads(df1.iloc[i].category)['slug'].split('/')
        list_category.append(category)
        list_subcategory.append(subcategory)
    except Exception as e:
        list_category.append(None)
        list_subcategory.append(None)

for i in range(len(df1)):
    try:
        loc_dict = json.loads(df1.iloc[i].location)
        country.append(str(loc_dict['country']))
    except Exception as e:
        country.append("None")
        
for i in range(len(df1)):
    try:
        loc_dict = json.loads(df1.iloc[i].location)
        state.append(str(loc_dict['state']))
    except Exception as e:
        state.append("None")
for i in range(len(df1)):
    try:
        loc_dict = json.loads(df1.iloc[i].location)
        town.append(str(loc_dict['name']))
    except Exception as e:
        town.append("None")
for i in range(len(df1)):
    try:
        loc_dict = json.loads(df1.iloc[i].location)
        loc_type.append(str(loc_dict['type']))
    except Exception as e:
        loc_type.append("None")

for i in range(len(df1)):
    try:
        project_url.append(str(json.loads(df1.iloc[i]['urls'])['web']['project']))
    except Exception as e:
        project_url.append("None")
for i in range(len(df1)):
    try:
        reward_url.append(str(json.loads(df1.iloc[i]['urls'])['web']['rewards']))
    except Exception as e:
        reward_url.append("None")        


df1['clean_category'] = np.array(list_category)
df1['subcategory'] = np.array(list_subcategory)
df1['photo_link'] = np.array(photo_link)
df1['clean_country'] = np.array(country)
df1['state'] = np.array(state)
df1['town'] = np.array(town)
df1['location_type'] = np.array(loc_type)
df1['project_url'] = project_url
df1['reward_url'] = reward_url

df1.drop(['category','urls','photo','location','profile', 'creator',
'currency_symbol',
'currency_trailing_code',
'current_currency',
'disable_communication', 
'fx_rate',
'friends',
'is_backing',
'is_starred'],axis=1,inplace=True)
df1.head()

,backers_count,blurb,converted_pledged_amount,country,created_at,currency,deadline,goal,id,is_starrable,...,usd_type,permissions,clean_category,subcategory,photo_link,clean_country,town,location_type,project_url,reward_url
0,22,"See some cool illustrations, Help make them av...",634,US,1361602103,USD,1367651035,600.0,1082237073,False,...,domestic,NaN,art,illustration,https://ksr-ugc.imgix.net/assets/011/485/371/d...,US,Lockport,Town,https://www.kickstarter.com/projects/690850738...,https://www.kickstarter.com/projects/690850738...
1,28,"High-quality 24""x36"" print parodying the ill-f...",1016,US,1364919898,USD,1367283600,1000.0,1574516491,False,...,domestic,NaN,art,illustration,https://ksr-ugc.imgix.net/assets/011/502/866/8...,US,Nashville,Town,https://www.kickstarter.com/projects/164500077...,https://www.kickstarter.com/projects/164500077...
2,8,This is a collection of 5 iconic New Orleans i...,945,US,1364070331,USD,1368040821,300.0,2080857761,False,...,domestic,NaN,art,illustration,https://ksr-ugc.imgix.net/assets/011/498/271/f...,US,New Orleans,Town,https://www.kickstarter.com/projects/174428992...,https://www.kickstarter.com/projects/174428992...
3,125,Award winning author and illustrator Shane W. ...,5949,US,1364365408,USD,1368881098,5000.0,802310873,False,...,domestic,NaN,art,illustration,https://ksr-ugc.imgix.net/assets/011/499/967/0...,US,Kansas City,Town,https://www.kickstarter.com/projects/olusdream...,https://www.kickstarter.com/projects/olusdream...
4,68,We'll illustrate you as a zombie and place you...,5539,GB,1366639145,GBP,1369699038,1200.0,1222458071,False,...,domestic,NaN,art,illustration,https://ksr-ugc.imgix.net/assets/011/511/971/7...,GB,North Yorkshire,County,https://www.kickstarter.com/projects/100515158...,https://www.kickstarter.com/projects/100515158...


In [8]:
df1.to_csv('cleaned_data001.csv')

In [ ]:
from bs4 import BeautifulSoup # documentation available at : #www.crummy.com/software/BeautifulSoup/bs4/doc/
from BeautifulSoup import NavigableString, Tag
import requests # To send http requests and access the page : docs.python-requests.org/en/latest/
import csv # To create the output csv file
import unicodedata # To work with the string encoding of the data

entries = []
entry = []
urlnumber = 2 # Give the page number to start with

while urlnumber<30: # Give the page number to end with
    
    #print type(urlnumber), urlnumber

    url = 'http://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p%d' % (urlnumber,) #Give the url of the forum, excluding the page number in the hyperlink
    #print url


    try:
        r = requests.get(url, timeout = 10) #Sending a request to access the page
    except Exception,e:
        break

    data = r.text

    soup = BeautifulSoup(data) # Getting the page source into the soup
    for div in soup.find_all('div'):
        entry = []
        if(div.get('class') != None and div.get('class')[0] == 'Comment'): # A single post is referred to as a comment. Each comment is a block denoted in a div tag which has a class called comment. 
            ps = div.find_all('p') #gets all the tags called p to a variable ps
            aas = div.find_all('a') # gets all the tags called a to a variable aas
            spans = div.find_all('span') #
            times = div.find_all('time') # used to extract the time tage which gives the iDate of the post
            
            concat_str = ''
            for str in aas[1].contents: # prints the contents that is between the tag start and end
                if str != "<br>" or str != "<br/>": # This denotes breaks in post which we need to work around.
                    concat_str = (concat_str + ' '+ str.encode('iso-8859-1')).strip() # The encoding is because the format exracted is a unicode. We need a uniform structure to work with the strings.
            entry.append(concat_str)
            
            concat_str = ''
            for str in times[0].contents:
                if str != "<br>" or str != "<br/>":
                    concat_str = (concat_str + ' '+ str.encode('iso-8859-1')).strip()
            entry.append(concat_str)

            #print "-------------------------"
            for div in div.find_all('div'):
                if (div.get('class') != None and div.get('class')[0] == 'Message'): # Extracting the div tag with the class attribute as message.
                    blockqoutes = []
                    x = div.get_text()
                    for bl in div.find_all('blockquote'):
                        blockqoutes.append(bl.get_text()) #Block quote is used to get the quote made by a person. get text helps to elimiate the hyperlinks and pulls out only the data. 
                        bl.decompose()
                

                    entry.append(div.get_text().replace("\n"," ").replace("<br/>","").encode('ascii','replace').encode('iso-8859-1'))


                    for bl in blockqoutes:
                        #print bl
                        entry.append(bl.replace("\n"," ").replace("<br/>","").encode('ascii','replace').encode('iso-8859-1'))
                
                #print entry
            entries.append(entry)

    urlnumber = urlnumber + 1 # increment so that we can extract the next page

with open('edmunds_extraction.csv', 'w') as output:
    writer = csv.writer(output, delimiter= ',', lineterminator = '\n')
    writer.writerows(entries)
print "Wrote to edmunds_extraction.csv"


In [3]:
entries = []
entry = []
urlnumber = 2 # Give the page number to start with


In [4]:
url = 'https://www.kickstarter.com/projects/692366438/upstate-photographs-by-tema-stauffer-daylight-book?ref=home_potd'


In [5]:
soup = BeautifulSoup(data) # Getting the page source into the soup


In [ ]:
from bs4 import BeautifulSoup # documentation available at : #www.crummy.com/software/BeautifulSoup/bs4/doc/
from BeautifulSoup import NavigableString, Tag
import requests # To send http requests and access the page : docs.python-requests.org/en/latest/
import csv # To create the output csv file
import unicodedata # To work with the string encoding of the data
cleaned_df = pd.read_csv('cleaned_data001.csv')
projects_url = cleaned_df['project_url'].tolist()
list_description =[]
list_rewards = []
    
for url in projects_url:
    try:
        r = requests.get(url, timeout = 10) #Sending a request to access the page
    except Exception,e:
        break
    data = r.text
    soup = BeautifulSoup(data) # Getting the page source into the soup
    ps = []
    for div in soup.find_all('div'):
        if (div.get('class') != None and div.get('class')[0] == 'full-description'):
            ps = div.find_all('p')
        if (div.get('class') != None and div.get('class')[0] == 'NS_projects__rewards_list'):
            li_len = len(div.find_all('li')) -1
        desc = ' ' 
        for i in ps:
            desc+= i.text
    list_rewards.append(li_len)
    list_description.append(desc.encode('utf-8'))


In [ ]:
cleaned_df['Description'] = np.array(list_description)
cleaned_df['Rewards'] = np.array(list_rewards)

In [16]:
projects_url

['https://www.kickstarter.com/projects/690850738/the-curious-concepts-project?ref=category_newest',
 'https://www.kickstarter.com/projects/164500077/et-the-extra-terrestrial-30th-anniversary-poster?ref=category_newest',
 'https://www.kickstarter.com/projects/1744289925/iconic-new-orleans-fine-art-prints?ref=category_newest',
 'https://www.kickstarter.com/projects/olusdream/face-of-a-dream-0?ref=category_newest',
 'https://www.kickstarter.com/projects/1005151588/go-zombie-zombie-portrait-print-project?ref=category_newest',
 'https://www.kickstarter.com/projects/2009141206/denver-comiccon-print-by-sobol-and-kordos?ref=category_newest',
 'https://www.kickstarter.com/projects/1579553529/immortalize-your-best-friend-in-a-great-1-of-a-kin?ref=category_newest',
 'https://www.kickstarter.com/projects/1317535290/lucid-dream-fantasy-portrait-art?ref=category_newest',
 'https://www.kickstarter.com/projects/325451491/eclecdeck-transformation-playing-cards?ref=category_newest',
 'https://www.kickst

In [98]:
from bs4 import BeautifulSoup # documentation available at : #www.crummy.com/software/BeautifulSoup/bs4/doc/
from BeautifulSoup import NavigableString, Tag
import requests # To send http requests and access the page : docs.python-requests.org/en/latest/
import csv # To create the output csv file
import unicodedata # To work with the string encoding of the data

 UPSTATE: Photographs by Tema Stauffer is a forthcoming hard cover book with thirty-three color photographs, a foreword by novelist Xhenet Aliu, and an essay by photo historian Alison Nordström. With your support, this monograph will be published by Daylight Books in fall 2018, coinciding with my New Faculty Solo Exhibition at the Reece Museum at East Tennessee State University opening on October 22nd, 2018. The photographs of UPSTATE explore the community, culture, landscape and architecture of one of the oldest regions in the country. Located on the shores of the upper Hudson River, the city of Hudson was the first city chartered in the United States in 1785 and it developed rapidly as a thriving whaling and merchant seaport. After an economic downturn in the early 19th century, Hudson’s economy rose again during the mid-century through heavy industries such as iron factories and mills. The Hudson River School, a mid-19th century art movement embodied by a group of landscape painters

In [99]:
type(desc.encode('utf-8'))


str